# Import Libraries

In [ ]:
import os
import sys
from tqdm import tqdm
import random
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pickle

import shutil
from matplotlib import pyplot as plt

import warnings; warnings.simplefilter('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = 30
np.set_printoptions(threshold=sys.maxsize)

# Add flood inundation to network nodes and edges

In [ ]:
# Flood the nodes with 10 flood return periods
RP_lst = [5, 10, 20, 50, 75, 100, 200, 250, 500, 1000]
for file in tqdm(os.listdir('L:/yiyi/grth_90_graphs_drive_service_no_res/')):
    net_id = int(file[15:][:-3])
    G = pickle.load(open('L:/yiyi/grth_90_graphs_drive_service_no_res/'+ file, 'rb'))
    G_flooded = G.copy()
    # Flood the nodes with 10 scenarios
    for rp in RP_lst:
        G_flooded = gn.sample_raster(G_flooded, 'J:/yiyi/mosaics/FUP'+ str(rp) +'_mosaic.tif', 'FUP_'+ str(rp))
    # Now we have flooded nodes with 10 flood depths
    # Flood the edges too by assigning the larger nodes flood depth to edge
    for i, j, data in G_flooded.edges.data():
        for rp in RP_lst:
            FUP_i = G_flooded.nodes[i]['FUP_'+str(rp)]
            FUP_j = G_flooded.nodes[j]['FUP_'+str(rp)]
            G_flooded[i][j][0]['FUP_'+str(rp)] = max(FUP_i, FUP_j)

    with open('L:/yiyi/grth_90_graphs_10flooded/G_cov_no_r_' + str(net_id) + '_10flooded.pk', 'wb') as handle:
                pickle.dump(G_flooded, handle, protocol=2)

# Remove nodes and edges based on flood inundation level

In [ ]:
RP_lst = [5, 10, 20, 50, 75, 100, 200, 250, 500, 1000]
problem_graph_ids = []
flooded_graphs_2616_dir = 'L:/yiyi/2616_flooded_graphs_drive_servce_no_res/'

for rp in RP_lst:
    print(rp)
    for flooded_graph_file in os.listdir(flooded_graphs_2616_dir):
        net_id = int(flooded_graph_file[11:][:-13])
        G_flooded =  pickle.load(open(flooded_graphs_2616_dir + flooded_graph_file, 'rb'))
        G_disrupted = G_flooded.copy()
        
        try:
            # Delete nodes with flood inundation greater than 30mm or 0.3m
            for node_id, node_data in G_flooded.nodes.data():
                if node_data['FUP_' + str(rp)] >= 0.3:
                    G_disrupted.remove_node(node_id)
            # Delete edges with flood inundation greater than 30mm or 0.3m
            for start_id, end_id, edge_data in G_flooded.edges.data():
                if edge_data['FUP_' + str(rp)] >= 0.3:
                    G_disrupted.remove_edge(start_id, end_id)
        except:
            problem_graph_ids.append(net_id)
        # Add original/dry speed
        try:
            G_disrupted_speed = ox.speed.add_edge_speeds(G_disrupted)
        except:
            problem_graph_ids.append(net_id)
            continue

        try:
            for start_id, end_id, edge_data in G_disrupted_speed.edges.data():
                edge_water_depth_mm = edge_data['FUP_' + str(rp)]
                theoretical_speed = 86.9448 - (0.5529*edge_water_depth_mm) + (0.0009*edge_water_depth_mm*edge_water_depth_mm)
                designed_speed = edge_data['speed_kph']
                G_disrupted_speed[start_id][end_id][0]['speed_kph'] = min(theoretical_speed, designed_speed)
            G_disrupted_speed_time = ox.speed.add_edge_travel_times(G_disrupted_speed)
        except:
             problem_graph_ids.append(net_id)

        with open('L:/yiyi/2616_disprupted_graphs/FUP_' + str(rp) + '/G_' + str(net_id) + '_disrupted_FUP_' + str(rp) + '.pk', 'wb') as handle:
            pickle.dump(G_disrupted_speed_time, handle, protocol=2)